## WSC Quiz Coin
#### 01 Smart Contracts Creation
##### Samuele Dell'Oca (samuele.delloca@supsi.ch)

The entire notebook is predisposed in such a way that the cells have to be executed in a sequential way for replicating the desired execution. In case some cells need to be skipped, there will be a proper indication.

For simplicity I have left also the code for interacting with the smart contract inside this notebook.


## Notebook Setup
Taken from the course examples. It loads:
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [1]:
# Loading shared code and credentials
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction, LogicSigAccount
import algosdk.error
import json
import base64

from pyteal import *
import random
import hashlib

In [3]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

### Quiz Coin creation

SKIP THE FOLLOWING CELL IF YOU DON'T WANT TO CREATE A NEW ASSET.

MyAlgo will be the creator of the asset and also the manager and reserve, for simplicity. Alice will have the power to freeze the asset and to clawback.
The total supply of the coin will be of 1'000'000, more than enough for running several quizzes. The coin will have 2 decimals. The unit name will be "SQC".

In [5]:
#  coin configuration
sp = algod_client.suggested_params()
token_supply = 1_000_000
token_decimals =  2
token_total = token_supply * 10**token_decimals

token_name  = "Samu WSC QuizCoin"
token_url   = "https://github.com/samueledelloca/WSC-quiz-coin"
token_unitname = "SQC"

#  asset creation
txn = AssetConfigTxn(
    sender=MyAlgo['public'],
    sp=sp,
    total=token_total,
    decimals=token_decimals,
    default_frozen=False,                      
    unit_name=token_unitname,                       
    asset_name=token_name,
    url=token_url,
    manager=MyAlgo['public'],                   # Special roles
    reserve=MyAlgo['public'],
    freeze=Alice['public'],
    clawback=Alice['public'],
)

#  signing and sending the transaction
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

txinfo = wait_for_confirmation(algod_client,txid)

#  asset id and explorer URL
asset_id = txinfo['asset-index']
print(asset_id)
print('https://testnet.algoexplorer.io/asset/{}'.format(asset_id))

3MPCI2C7LG5MDR7MPOCLMNWMHWI5DLQ5V2D5IYKWUNCLUBEKPQLQ
Current round is  28938224.
Waiting for round 28938224 to finish.
Waiting for round 28938225 to finish.
Transaction 3MPCI2C7LG5MDR7MPOCLMNWMHWI5DLQ5V2D5IYKWUNCLUBEKPQLQ confirmed in round 28938226.


In [5]:
#  in case you skipped the previous cell, that's the asset ID of the official Quiz Coin
asset_id = 175523481

In [6]:
#  check MyAlgo wallet, all the initial supply will be in its possession, if the Quiz Coin has been just created.
asset_holdings_df(algod_client, MyAlgo['public'])

,amount,unit,asset-id,name,decimals
0,130.717,ALGO,0,Algorand,6
1,10.000,USDC,10458941,USDC,6
2,73.200,FNS,159159590,Finus WSC coin,2
3,973.570,WSC,159159603,Samu WSC coin,2
4,1000.000,SWBC,159489951,Samu WSC Beer Coin,1
5,1.000,LUG1,159493679,Lugano NFT,0
6,999950.000,SQC,175523481,Samu WSC QuizCoin,2


### Dispenser Delegated Smart Signature

I have chosen to use a delegated Smart Signature since dispensing funds is a common operation and it does not require to manually sign the transactions each time. I didn't need a proper Smart Contract for dispensing the Quiz Coins, the Smart Signature is enough.
I expect 2 transactions, hence I use a transaction group: the first will be the amount of Algo to pay, in this case 0 since it is a free dispenser, and the second one will be the amount of Quiz Coins to withdraw from the Smart Signature.
I could have put a limit on the amount of Quiz Coins that one can request, but since I will fund the dispenser with a limited quantity each time I need it, a big request of Quiz Coins is not really a problem, since it will fail for insufficient funds in the Smart Signature.

In [33]:
#  conditions as a PyTeal program

dispense_condition = And(
    Global.group_size() == Int(2),  #  atomic swap with 2 transactions
    Gtxn[0].type_enum() == TxnType.Payment,  #  the first transaction is a payment in ALGOs
    Gtxn[0].xfer_asset() == Int(0),
    Gtxn[1].type_enum() == TxnType.AssetTransfer,  #  the second transaction is an asset transfer in Quiz Coins
    Gtxn[1].xfer_asset() == Int(asset_id),
    Int(int(1 * 1E6)) * Gtxn[0].amount() == Int(0), #  the amount of ALGO has to be 0, since this is a free dispenser
    #  security conditions
    Gtxn[0].receiver() == Gtxn[1].sender(),
    Gtxn[0].sender() == Gtxn[1].asset_receiver(),
    Gtxn[0].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[1].asset_close_to() == Global.zero_address(),
)

optin_condition = And(
    Global.group_size() == Int(1),
    Txn.type_enum() == TxnType.AssetTransfer,
    Txn.asset_amount() == Int(0),
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address(),
)

a = Int(random.randrange(2 ** 32 - 1))
random_cond = (a == a)

fee_cond = Txn.fee() <= Int(1000)

dispense_pyteal = And(
    random_cond,
    fee_cond,
    If(
        Global.group_size() == Int(1),  # only admitted transactions are opt-in (1 transaction) and dispense requests (2 transactions)
        optin_condition,
        dispense_condition
    )
)

In [34]:
#  compilation of the Dispense program

dispense_teal = compileTeal(dispense_pyteal, Mode.Signature, version=8)
Dispense = algod_client.compile(dispense_teal)
print("Smart signature addr: ", Dispense['hash'])
print("Smart signature code: ", Dispense['result'])

#  dispense funding: min holdings + min holdings for 1 ASA + TX fee for 50 swaps
sp = algod_client.suggested_params()
amt = int(0.1 * 1e6) + int(0.1 * 1e6) + int(50 * 0.001 * 1e6)
txn = transaction.PaymentTxn(sender=MyAlgo['public'], sp=sp,
                             receiver=Dispense['hash'], amt=amt)

#  funding signing and sending
stxn = txn.sign(MyAlgo['private'])

txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

#  Smart Contract asset opt-in
sp = algod_client.suggested_params()
txn = AssetTransferTxn(Dispense['hash'], sp, Dispense['hash'], 0, asset_id)

#  opt-in signing and sending
encodedProg = Dispense['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Smart signature addr:  YVGRKP255O2MRGK4P6UVTQU2W4DNZSOMI4PXPLEQSN5WV7GJEIVDHE4QGQ
Smart signature code:  CCAEAQCL7fyKDAQkJBIxAYHoBw4QMgQiEkAAWzIEgQISMwAQIhIQMwARIxIQMwEQJRIQMwERgZmN2VMSEIHAhD0zAAgLIxIQMwAHMwEAEhAzAAAzARQSEDMAIDIDEhAzAAkyAxIQMwEgMgMSEDMBFTIDEhBCABoyBCISMRAlEhAxEiMSEDEgMgMSEDEJMgMSEBBD
Current round is  29025454.
Waiting for round 29025454 to finish.
Waiting for round 29025455 to finish.
Transaction AKMJODXBOVULJVXLC5E4CUOMDE4AZ6COIOKV5EUZ5RDHSRNQNZ6Q confirmed in round 29025456.
Current round is  29025456.
Waiting for round 29025456 to finish.
Waiting for round 29025457 to finish.
Transaction NA665XH2OITI24DBNVFFTM3OL6S6ZTNOMUB5LBUOCYE6SWNWTEXA confirmed in round 29025458.


In [15]:
#  putting 50 Quiz Coin inside the dispenser
asset_decimals = algod_client.asset_info(asset_id)['params']['decimals']
amt = int(50 * 10 ** asset_decimals)

sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Dispense['hash'],
    amt=amt,
    index=asset_id)

#  signing and sensing
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025358.
Waiting for round 29025358 to finish.
Waiting for round 29025359 to finish.
Transaction D2T7G5J4CA25UB7FMAH6BQ67VV2JPZILMWX3P3HZR6IYSXKPNLHQ confirmed in round 29025360.


In [16]:
#  Bob (a user) needs to opt into the Quiz Coin before participating.

sp = algod_client.suggested_params()
amt = int(0)

txn = AssetTransferTxn(
    sender=Bob['public'],
    sp=sp,
    receiver=Bob['public'],
    amt=amt,
    index=asset_id
)

#  opt-in signing and sending
stxn = txn.sign(Bob['private'])
txid = algod_client.send_transaction(stxn)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025360.
Waiting for round 29025360 to finish.
Transaction PMCPIEWW6GNVRLVBVADJLDZ4O4HRODQWES4YJXDAPBGU2X3IPGNQ confirmed in round 29025362.


In [17]:
#  the Dispenser is ready to use. Bob gets 5 Quiz Coins.

sp = algod_client.suggested_params()

#  ALGO payment from Bob to Dispenser
amt_1 = int(0.0 * 1e6)
txn_1 = PaymentTxn(Bob['public'], sp, Dispense['hash'], amt_1)

#  Quiz Coin asset transfer from Dispenser to Bob
amt_2 = int(5 * 10**asset_decimals)
txn_2 = AssetTransferTxn(Dispense['hash'], sp, Bob['public'], amt_2, asset_id)

#  transaction group creation
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid  # add group_id to each transaction
txn_2.group = gid

#  Bob signs the first transaction, Smart signature will authorize the second one
stxn_1 = txn_1.sign(Bob['private'])
encodedProg = Dispense['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

#  transactions sending
signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025384.
Waiting for round 29025384 to finish.
Waiting for round 29025385 to finish.
Transaction CYM32A6RW7IXHQX2MVY57ENHWADJKVL2KZO4IAYUEYUMDPM4KAUQ confirmed in round 29025386.


### Question and answer

Now that the Quiz Coins can be received to the dispenser, is time to create the Quiz. I decided to use again a Smart signature with a password which unlocks the prize: the password corresponds to the answer. A user has to send exactly 1 Quiz Coin for partecipating to the Quiz. In this case the question is "In which Country is Lugano Located?".
The users have a week of time to submit their answer. After a week submissions are not admitted anymore, and the Quiz Company (MyAlgo) will retrieve the prize, if still available. If the Quiz get answered, further answers will result in a failure, since there won't be money in the Smart Signature anymore.

In [35]:
#  Quiz program definition

secret_password = 'Switzerland'
pwdhash = hashlib.sha256(secret_password.encode())

start_round = algod_client.status()["last-round"]

#  withdraw condition: only MyAlgo (the Quiz Company) can withdraw funds from the Smart Contract, if 1 week is passed
withdraw_cond = And (
    Gtxn[0].type_enum() == TxnType.AssetTransfer,  # the first transaction is an asset transfer for the Quiz Coin
    Gtxn[0].xfer_asset() == Int(asset_id),
    Gtxn[0].asset_receiver() == Addr(MyAlgo["public"]),
    Gtxn[1].type_enum() == TxnType.Payment,  # the second transaction is an ALGO payment for the prize
    Gtxn[1].xfer_asset() == Int(0),
    Gtxn[1].receiver() == Addr(MyAlgo["public"])
)

q_and_a_condition = And(
    Global.group_size() == Int(2),  # 2 transactions: 1 for getting the Quiz Coin and 1 for submitting the prize
    Gtxn[0].type_enum() == TxnType.AssetTransfer,  # the first transaction is an asset transfer for the Quiz Coin
    Gtxn[0].xfer_asset() == Int(asset_id),
    Gtxn[1].type_enum() == TxnType.Payment,  # the second transaction is an ALGO payment for the prize
    Gtxn[1].xfer_asset() == Int(0),
    Gtxn[0].asset_amount() == Int(int(1 * 1E2)), # receive exactly 1 Quiz Coin
    Sha256(Gtxn[0].note()) == Bytes("base16", pwdhash.hexdigest()),  # check if the answer is correct
    # security checks
    Gtxn[0].asset_receiver() == Gtxn[1].sender(),
    Gtxn[0].sender() == Gtxn[1].receiver(),
    Gtxn[0].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[1].asset_close_to() == Global.zero_address(),
)

vending_condition = (
    If(
        Txn.first_valid() < Int(start_round + 201600),  #  quiz closure after 1 week
        # Txn.first_valid() < Int(start_round + 10),  # 30 seconds instead of 1 week only for testing
        q_and_a_condition,  # if the Quiz is still valid, answers are accepted
        withdraw_cond  #  otherwise the company can withdraw the funds
    )
)

optin_condition = And(
    Global.group_size() == Int(1),
    Txn.type_enum() == TxnType.AssetTransfer,
    Txn.asset_amount() == Int(0),
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address(),
)

a = Int(random.randrange(2 ** 32 - 1))
random_cond = (a == a)

fee_cond = Txn.fee() <= Int(1000)

quiz_pyteal = And(
    random_cond,
    fee_cond,
    If(
        Global.group_size() == Int(1),
        optin_condition,
        vending_condition
    )
)

In [36]:
#  Quiz program compilation
quiz_teal = compileTeal(quiz_pyteal, Mode.Signature, version=8)

Quiz = algod_client.compile(quiz_teal)
print("Smart signature addr: ", Quiz['hash'])
print("Smart signature code: ", Quiz['result'])

Smart signature addr:  VRBJLYDADJWW6NF7NEDTG6343LGUYZFPSCGVL374ZXFZEYHGDDPISQXI7M
Smart signature code:  CCAFAQQAut/vswGZjdlTJgEgmLUFmOFjnZBD7fj7YFWjJ9+GwqzlS0hj1j21yzBGIgglJRIxAYHoBw4QMgQiEkAArjECgZnx9w0MQAAnMwAQIxIzABEhBBIQMwAUKBIQMwEQIhIQMwERJBIQMwEHKBIQQgCWMgSBAhIzABAjEhAzABEhBBIQMwEQIhIQMwERJBIQMwASgWQSEDMABQGAICJ1WDGW15FAWJKqyg2HdDyHLz/AzzMIpsPvglKJGKqKEhAzABQzAQASEDMAADMBBxIQMwAgMgMSEDMACTIDEhAzASAyAxIQMwEVMgMSEEIAGjIEIhIxECMSEDESJBIQMSAyAxIQMQkyAxIQEEM=


In [20]:
#  Smart Signature funding
sp = algod_client.suggested_params()
amt = int(5.1 * 1e6)
txn = transaction.PaymentTxn(sender=MyAlgo['public'], sp=sp, receiver=Quiz['hash'], amt=amt)

#  transaction signing and sending
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025391.
Waiting for round 29025391 to finish.
Waiting for round 29025392 to finish.
Waiting for round 29025393 to finish.
Transaction FA2JWZLZ5Z7VRWXOFXPRBTAXTMXCE2ZBUQ7GSWSIYZHXONIRLAWA confirmed in round 29025394.


In [22]:
#  Quiz Smart Signature asset opt-in (into Quiz Coin)
sp = algod_client.suggested_params()
txn = AssetTransferTxn(Quiz['hash'], sp, Quiz['hash'], 0, asset_id)

#  transaction signing and sending
encodedProg = Quiz['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

txid = algod_client.send_transaction(stxn)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025401.
Waiting for round 29025401 to finish.
Waiting for round 29025402 to finish.
Transaction XJSV2JXF4YM3QZ3ICOICETZ5GLGYGT4VLCNI7OOVM7LEAGUUWHHQ confirmed in round 29025403.


In [24]:
#  Quiz answer submission

sp = algod_client.suggested_params()

my_note = 'Switzerland'
note = my_note.encode()

#  Quiz Coin asset transfer from Bob to Smart Signature
amt_1 = int(1.0 * 1e2)
txn_1 = AssetTransferTxn(Bob['public'], sp, Quiz['hash'], amt_1, asset_id, note=note)

#  Algo prize transfer from Smart Sigature to Bob
# amt_2 = int(50.0 * 1e6)
amt_2 = int(1.0 * 1e6)  # it should be 50, I've used lower quantities for testing
txn_2 = PaymentTxn(Quiz['hash'], sp, Bob['public'], amt_2)

#  transaction group creation, signing and sending
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

stxn_1 = txn_1.sign(Bob['private'])
encodedProg = Quiz['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025409.
Waiting for round 29025409 to finish.
Waiting for round 29025410 to finish.
Transaction 2RIO7CZ6UHN6UMKGYMN2E4YCH227VCVSDZSOYZZAZSLEGD7ZPKBA confirmed in round 29025411.


In [54]:
#  money retrieving from Quiz Company after quiz closure (1 week). The company retrieves both the Quiz Coins (first transaction) and the ALGOs (second transaction).

sp = algod_client.suggested_params()

#  Quiz Coins retrieving from Smart Signature
amt_1 = int(1.0 * 1e2) #  it should be set based on the amount of Quiz Coins
txn_1 = AssetTransferTxn(Quiz['hash'], sp, MyAlgo['public'], amt_1, asset_id)

#  ALGOs retrieving from Smart Signature
amt_2 = int(0.1 * 1e6)  #  it should be set based on the amount of ALGOs
txn_2 = PaymentTxn(Quiz['hash'], sp, MyAlgo['public'], amt_2)

#  transaction group creation, signing and sending
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

encodedProg = Quiz['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_1 = LogicSigTransaction(txn_1, lsig)
stxn_2 = LogicSigTransaction(txn_2, lsig)

signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

AlgodHTTPError: TransactionPool.Remember: transaction BD4A53KEYRYDM7EAPUPXIDJCMPQUVQPDKVWIKWY67PCEFXPCLGJA: underflow on subtracting 100 from sender amount 0

In [25]:
#  check the asset possession of Bob after winning the Quiz

asset_holdings_df(algod_client, Bob['public'])

,amount,unit,asset-id,name,decimals
0,12.005004,ALGO,0,Algorand,6
1,10.000000,USDC,10458941,USDC,6
2,0.050000,WSC,159159603,Samu WSC coin,2
3,13.350000,SQC,175523481,Samu WSC QuizCoin,2


### Quiz Coin Selling 1:1

An alternative to the Dispenser is the sell of Quiz Coins. The following program will sell Quiz Coins at a fixed ratio of 1 ALGO per Quiz Coin.

In [37]:
#  1:1 Quiz Coin selling program

vending_condition = And(
    Global.group_size() == Int(2),  #  vending atomic swap with 2 transactions
    Gtxn[0].type_enum() == TxnType.Payment,  #  the first transaction is a payment in ALGO
    Gtxn[0].xfer_asset() == Int(0),
    Gtxn[1].type_enum() == TxnType.AssetTransfer,  #  the second transaction is an asset transfer in Quiz Coins
    Gtxn[1].xfer_asset() == Int(asset_id),
    Int(int(1 * 1E2)) * Gtxn[0].amount() == Int(int(1 * 1E6)) * Gtxn[1].asset_amount(),  # 1:1 ratio
    # security checks
    Gtxn[0].receiver() == Gtxn[1].sender(),
    Gtxn[0].sender() == Gtxn[1].asset_receiver(),
    Gtxn[0].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[1].asset_close_to() == Global.zero_address(),
)

optin_condition = And(
    Global.group_size() == Int(1),
    Txn.type_enum() == TxnType.AssetTransfer,
    Txn.asset_amount() == Int(0),
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address(),
)

a = Int(random.randrange(2 ** 32 - 1))
random_cond = (a == a)

fee_cond = Txn.fee() <= Int(1000)

vending_pyteal = And(
    random_cond,
    fee_cond,
    If(
        Global.group_size() == Int(1),
        optin_condition,
        vending_condition
    )
)

### Quiz Coin Selling with rebate scheme

SKIP THE FOLLOWING CELL AND USE THE 1:1 RATIO

Another alternative to the Dispenser is the sell of Quiz Coins with a rebate scheme. There is something wrong with computing the discount, so the solution is not entirely working.

In [38]:
#  rebate scheme selling program

vending_condition = And(
    Global.group_size() == Int(2),  #  vending atomic swap with 2 transactions
    Gtxn[0].type_enum() == TxnType.Payment,  #  the first transaction is a payment in ALGO
    Gtxn[0].xfer_asset() == Int(0),
    Gtxn[1].type_enum() == TxnType.AssetTransfer,  #  the second transaction is an asset transfer in Quiz Coins
    Gtxn[1].xfer_asset() == Int(asset_id),

    If (
        Gtxn[0].amount() < Int(int(10 * 1E6)),
        Int(int(1 * 1E2)) * Gtxn[0].amount() == Int(int(1 * 1E6)) * Gtxn[1].asset_amount(),  # no discount
        If (
            Gtxn[0].amount() < Int(int(50 * 1E6)),
            Int(int(1 * 1E2)) * (Gtxn[0].amount() / Int(int(10)) * Int(int(9))) == Int(int(1 * 1E6)) * Gtxn[1].asset_amount(),  # 10% discount
            Int(int(1 * 1E2)) * (Gtxn[0].amount() / Int(int(10)) * Int(int(8))) == Int(int(1 * 1E6)) * Gtxn[1].asset_amount(),  # 20% discount
        )
    ),

    # security checks
    Gtxn[0].receiver() == Gtxn[1].sender(),
    Gtxn[0].sender() == Gtxn[1].asset_receiver(),
    Gtxn[0].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[1].asset_close_to() == Global.zero_address(),
)

optin_condition = And(
    Global.group_size() == Int(1),
    Txn.type_enum() == TxnType.AssetTransfer,
    Txn.asset_amount() == Int(0),
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address(),
)

a = Int(random.randrange(2 ** 32 - 1))
random_cond = (a == a)

fee_cond = Txn.fee() <= Int(1000)

vending_pyteal = And(
    random_cond,
    fee_cond,
    If(
        Global.group_size() == Int(1),
        optin_condition,
        vending_condition
    )
)

In [39]:
#  vending program compilation

vending_teal = compileTeal(vending_pyteal, Mode.Signature, version=8)
Vending = algod_client.compile(vending_teal)
print("Smart signature addr: ", Vending['hash'])
print("Smart signature code: ", Vending['result'])

#  Smart Signature funding: min holdings + min holdings for 1 ASA + TX fee for a few several swaps
sp = algod_client.suggested_params()
amt = int(0.1 * 1e6) + int(0.1 * 1e6) + int(50 * 0.001 * 1e6)
txn = transaction.PaymentTxn(sender=MyAlgo['public'], sp=sp,
                             receiver=Vending['hash'], amt=amt)

#  funding signing and sending
stxn = txn.sign(MyAlgo['private'])

txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

#  Smart Contract asset opt-in
sp = algod_client.suggested_params()
txn = AssetTransferTxn(Vending['hash'], sp, Vending['hash'], 0, asset_id)

#  opt-in signing and sending
encodedProg = Vending['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Smart signature addr:  2OD3S53WDKSYJIFPDHC5SJIPZBDYJ2BS4UJJP7QYSS3OGPPVRGPO5DL52I
Smart signature code:  CCAHAWTAhD3l/I5uAAQKJSUSMQGB6AcOEDIEIhJAAJ4yBIECEjMAECISEDMAESEEEhAzARAhBRIQMwERgZmN2VMSEDMACIGAreIEDEAAYTMACIGA4esXDEAAQSMzAAghBgqBCAsLJDMBEgsSEDMABzMBABIQMwAAMwEUEhAzACAyAxIQMwAJMgMSEDMBIDIDEhAzARUyAxIQQgA+IzMACCEGCoEJCwskMwESCxJC/7wjMwAICyQzARILEkL/rjIEIhIxECEFEhAxEiEEEhAxIDIDEhAxCTIDEhAQQw==
Current round is  29025595.
Waiting for round 29025595 to finish.
Waiting for round 29025596 to finish.
Transaction A3RH6PBHUADLYOJTZJQUTEJKKQVIVIKG6PTKKPWQ6KDFZ2BHX3TA confirmed in round 29025597.
Current round is  29025597.
Waiting for round 29025597 to finish.
Waiting for round 29025598 to finish.
Transaction FQ7DCOUFRWOAKLJJIGKLHENAGHRBORX6KRS5B6ZJNF7DH4QWE4MA confirmed in round 29025599.


In [31]:
#  putting 50 Quiz Coin inside the vending contract
asset_decimals = algod_client.asset_info(asset_id)['params']['decimals']
amt = int(50 * 10 ** asset_decimals)

sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Vending['hash'],
    amt=amt,
    index=asset_id)

#  signing and sensing
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025436.
Waiting for round 29025436 to finish.
Waiting for round 29025437 to finish.
Transaction 2LUST33OUPHW7FNX4KXV6XF76YJGXX6LJAXK3TVI4VVUYPNHWTRA confirmed in round 29025438.


In [32]:
##  using the selling contract

sp = algod_client.suggested_params()

#  payment transaction in ALGO from Bob to Vending
amt_1 = int(5.0 * 1e6)
txn_1 = PaymentTxn(Bob['public'], sp, Vending['hash'], amt_1)

#  asset transfer transaction from Vending to Bob
amt_2 = int(5.0 * 1e2)
txn_2 = AssetTransferTxn(Vending['hash'], sp, Bob['public'], amt_2, asset_id)

#  transaction group creation, signing and sending
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

stxn_1 = txn_1.sign(Bob['private'])
encodedProg = Vending['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

Current round is  29025439.
Waiting for round 29025439 to finish.
Waiting for round 29025440 to finish.
Transaction S3T3JDIVHPOK5J7PI3YK2P5ZZHR63TDEA7RLIFFENB7TBDPC2RVA confirmed in round 29025441.


In [91]:
#  checking the wallet of Bob to see the acquired assets

asset_holdings_df(algod_client, Bob['public'])

,amount,unit,asset-id,name,decimals
0,18.011004,ALGO,0,Algorand,6
1,10.000000,USDC,10458941,USDC,6
2,0.050000,WSC,159159603,Samu WSC coin,2
3,2.350000,SQC,175523481,Samu WSC QuizCoin,2
